#LSH



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# !wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
# !cp drive/MyDrive/NCKH_2023-2024/spark-3.1.1-bin-hadoop3.2.tgz .
!cp drive/MyDrive/NCKH_2023-2024/spark-3.1.1-bin-hadoop3.2.tgz .
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
  import os
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
  os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [ ]:
!echo $SPARK_HOME

/content/spark-3.1.1-bin-hadoop3.2


In [ ]:
# ! $SPARK_HOME/bin/pyspark

In [ ]:
import findspark
findspark.init()
print(findspark.__version__)

2.0.1


In [ ]:
import pyspark as spark
print(spark.__version__)

3.1.1


In [ ]:


import numpy as np
import os
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.ml.feature import BucketedRandomProjectionLSH

spark = SparkSession.builder \
    .appName("Numpy to DataFrame") \
    .getOrCreate()

def read_npy_files_from_parent_dir(parent_dir):
    schema = StructType([
        StructField("id", StringType(), nullable=False),
        StructField("features", VectorUDT(), nullable=False)
    ])

    def process_file(file, idx):
        file_name = os.path.basename(file)
        vector = np.load(file)
        vector = Vectors.dense(*vector.tolist())  # Chuyển đổi sang Vector của PySpark
        return (file_name, vector)

    all_files = []
    for root, dirs, files in os.walk(parent_dir):
        for idx, file in enumerate(files):
            if file.endswith(".npy"):
                file_path = os.path.join(root, file)
                all_files.append((file_path, idx))

    rdd = spark.sparkContext.parallelize(all_files)
    df = rdd.map(lambda x: process_file(x[0], x[1])).toDF(schema)
    return df
def count_similar_ids(dataframe, id_column, length=10):
    dataframe['id'] = dataframe[id_column].str[:length]
    counts = dataframe['id'].value_counts().reset_index()
    counts.columns = ['id', 'count']
    return counts
directory = "/content/drive/MyDrive/NCKH_2023-2024/vectorEmbeddingToTrainLSH"
# directory = "/content/drive/Shareddrives/NCKH/NCKH_2023-2024/vectorEmbeddingToTrainLSH"
directory_test = "/content/drive/MyDrive/NCKH_2023-2024/imageTestLSH"
df = read_npy_files_from_parent_dir(directory)
df1 = read_npy_files_from_parent_dir(directory_test)

#create model
brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", seed=12345, bucketLength=2.0)
#train model
model = brp.fit(df)
# # save model
# model_path = "/content/drive/Shareddrives/NCKH/NCKH_2023-2024/model_lsh/"
# model.save(model_path)

#transformed
transformed = model.transform(df)
transformed.show()

# random vector df
random_row = df1.sample(False, 0.1, seed=42).first()
random_id = random_row["id"]
random_vector = random_row["features"]
# in ra vector test
print("Random ID:", random_id)
print("Random Vector:")
print(random_vector)

# Tìm dữ liệu gần nhất với vector ngẫu nhiên
nearest_df = model.approxNearestNeighbors(df, random_vector, 10)
nearest_df.show()

# spark.stop()


+--------------------+--------------------+--------+
|                  id|            features|  hashes|
+--------------------+--------------------+--------+
|Teri_Hatcher_7360...|[0.0,0.0,2.109123...|[[-1.0]]|
|Teri_Hatcher_7387...|[0.0,0.0,0.337369...|[[-1.0]]|
|Teri_Hatcher_7367...|[0.0,0.0,0.543933...|[[-1.0]]|
|Teri_Hatcher_7373...|[0.0,0.1559111028...|[[-1.0]]|
|Teri_Hatcher_7369...|[0.20337395370006...|[[-1.0]]|
|Teri_Hatcher_7377...|[0.0,0.0,2.559999...|[[-1.0]]|
|Teri_Hatcher_7372...|[0.01275208685547...|[[-1.0]]|
|Teri_Hatcher_7377...|[0.0,0.0,0.272537...|[[-1.0]]|
|Teri_Hatcher_7379...|[0.0,0.0,0.133977...|[[-1.0]]|
|Teri_Hatcher_7383...|[1.02811241149902...| [[0.0]]|
|Katherine_Kelly_L...|[0.42808580398559...| [[0.0]]|
|Katherine_Kelly_L...|[0.05348702520132...| [[0.0]]|
|Katherine_Kelly_L...|[0.0,0.0760117322...| [[0.0]]|
|Katherine_Kelly_L...|[0.10087862610816...| [[0.0]]|
|Katherine_Kelly_L...|[0.0,0.0,1.184712...| [[0.0]]|
|Katherine_Kelly_L...|[0.0,0.0,0.296601...| [[

In [ ]:
import pandas as pd
df = nearest_df.toPandas()

def count_similar_ids(dataframe, id_column, length=10):
    dataframe['id'] = dataframe[id_column].str[:length]
    counts = dataframe['id'].value_counts().reset_index()
    counts.columns = ['id', 'count']
    return counts

# Sử dụng hàm để đếm các id giống nhau từ 10 ký tự đầu tiên
result = count_similar_ids(df, 'id')
print(result)

           id  count
0  Adam_McKay      6
1  David_Cros      2
2  Joe_Pantol      1
3  Paul_Betta      1
